In [ ]:
imUsingColab = False
shouldDownload = True # Set this to False after the first run to avoid redownloading the dataset
if shouldDownload:
  if imUsingColab:
    # Generate the .env file within the colab environment programatically
    file = open('.env','w') 
    file.write('KAGGLE_USERNAME=') # Populate this with your KAGGLE_USERNAME DO NOT COMMIT!!
    file.write('\nKAGGLE_KEY=') # Populate this with your KAGGLE api Key DO NOT COMMIT!!
    file.close()
    # Fetch the downloadData dependency 
    import urllib.request
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/lihaojin/Movie_Data_Analysis/master/downloadData.py", "downloadData.py")

  import downloadData
  downloadData.download(imUsingColab)


In [20]:
%run datasets.py
%run functions.py

import pandas as pd
import numpy as np
import math, nltk, warnings
from nltk.corpus import wordnet
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz

warnings.filterwarnings('ignore')

In [7]:
df = mergeData()
df.head()

,movieId,keywords,belongs_to_collection,budget,original_language,overview,popularity,revenue,runtime,status,...,year,month,day,genres,directors,actor1,actor2,actor3,actor4,actor5
0,862,jealousy|toy|boy|friendship|friends|rivalry|bo...,Toy Story Collection,30000000,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,Released,...,1995.0,10.0,30.0,Animation|Comedy|Family,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn
1,8844,board game|disappearance|based on children's b...,NaN,65000000,en,When siblings Judy and Peter discover an encha...,17.015539,262797249.0,104.0,Released,...,1995.0,12.0,15.0,Adventure|Fantasy|Family,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,Bonnie Hunt
2,15602,fishing|best friend|duringcreditsstinger|old men|,Grumpy Old Men Collection,0,en,A family wedding reignites the ancient feud be...,11.712900,NaN,101.0,Released,...,1995.0,12.0,22.0,Romance|Comedy,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,Daryl Hannah
3,31357,based on novel|interracial relationship|single...,NaN,16000000,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156.0,127.0,Released,...,1995.0,12.0,22.0,Comedy|Drama|Romance,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon,Gregory Hines
4,11862,baby|midlife crisis|confidence|aging|daughter|...,Father of the Bride Collection,0,en,Just when George Banks has recovered from his ...,8.387519,76578911.0,106.0,Released,...,1995.0,2.0,10.0,Comedy,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley,George Newbern


In [8]:
set_keywords = set()
for list_keywords in df['keywords'].str.split('|').values:
    if isinstance(list_keywords, float): continue  # only happen if list_keywords = NaN
    set_keywords = set_keywords.union(list_keywords)

In [9]:
keyword_occurences, dum = count_word(df, 'keywords', set_keywords)
keyword_occurences[1:6]

[['woman director', 2699],
 ['independent film', 1865],
 ['murder', 1278],
 ['based on novel', 817],
 ['musical', 716]]

In [10]:
df['genres'] = df['genres'].astype(str)
genre_labels = set()
for s in df['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [11]:
keyword_occurences, dum = count_word(df, 'genres', genre_labels)
keyword_occurences[:5]

[['Drama', 19974],
 ['Comedy', 12921],
 ['Thriller', 7551],
 ['Romance', 6658],
 ['Action', 6527]]

In [12]:
#doubled_entries = df_initial[df_initial.duplicated()]
doubled_entries = df[df.movieId.duplicated()]
doubled_entries.shape

(29, 27)

In [13]:
df_temp = df

In [14]:
list_var_duplicates = ['title', 'year', 'directors']

In [15]:
list_duplicates = df_temp['title'].map(df_temp['title'].value_counts() > 1)
print("Nb. of duplicate entries: {}".format(
    len(df_temp[list_duplicates][list_var_duplicates])))

Nb. of duplicate entries: 5243


In [16]:
#df_temp[list_duplicates][list_var_duplicates].sort_values('movie_title')[31:41]
df_temp[list_duplicates][list_var_duplicates].sort_values('title')

,title,year,directors
23444,10 Minutes,2002.0,Ahmed Imamović
40132,10 Minutes,2014.0,Lee Yong-Seung
1137,12 Angry Men,1957.0,Sidney Lumet
14883,12 Angry Men,1997.0,William Friedkin
31252,12 Chairs,1976.0,Mark Zakharov
31265,12 Chairs,1971.0,Leonid Gayday
23167,"20,000 Leagues Under the Sea",1997.0,Michael Anderson
971,"20,000 Leagues Under the Sea",1954.0,Richard Fleischer
12776,"20,000 Leagues Under the Sea",1916.0,Stuart Paton
20127,"20,000 Leagues Under the Sea",1997.0,Rod Hardy


In [17]:
keywords, keywords_roots, keywords_select = keywords_inventory(df, column = 'keywords')

Nb of keywords in variable 'keywords': 18871


In [18]:
# Plot of a sample of keywords that appear in close varieties 
#------------------------------------------------------------
icount = 0
for s in keywords_roots.keys():
    if len(keywords_roots[s]) > 1: 
        icount += 1
        if icount < 15: print(icount, keywords_roots[s], len(keywords_roots[s]))

1 {'toys', 'toy'} 2
2 {'boy', 'boys'} 2
3 {'friend', 'friends'} 2
4 {'fish', 'fishing'} 2
5 {'best friends', 'best friend'} 2
6 {'divorce', 'divorced'} 2
7 {'confidence', 'confidant'} 2
8 {'banking', 'bank'} 2
9 {'murderer', 'murders', 'murder'} 3
10 {'suspension', 'suspense'} 2
11 {'betrayal', 'betrayed'} 2
12 {'gangs', 'gang'} 2
13 {'explosions', 'explosives', 'explosion', 'explosive'} 4
14 {'widower', 'widow'} 2


In [21]:
df_keywords_cleaned = replacement_df_keywords(df, keywords_select,
                                               roots = True)

In [24]:
# Count of the keywords occurences
#----------------------------------
keyword_occurences, keywords_count = count_word(df_keywords_cleaned,'keywords',keywords)
keyword_occurences[1:6]

[['woman director', 2699],
 ['independent film', 1865],
 ['murder', 1308],
 ['music', 907],
 ['based on novel', 817]]

In [25]:
# Exemple of a list of synonyms given by NLTK
#---------------------------------------------------
mot_cle = 'alien'
lemma = get_synonyms(mot_cle)
for s in lemma:
    print(' "{:<30}" in keywords list -> {} {}'.format(s, s in keywords,
                                                keywords_count[s] if s in keywords else 0 ))

 "extraterrestrial being        " in keywords list -> False 0
 "noncitizen                    " in keywords list -> False 0
 "unknown                       " in keywords list -> False 0
 "alien                         " in keywords list -> True 289
 "extraterrestrial              " in keywords list -> True 13
 "outlander                     " in keywords list -> False 0
 "stranger                      " in keywords list -> True 30
 "foreigner                     " in keywords list -> False 0


In [26]:
keyword_occurences.sort(key = lambda x:x[1], reverse = False)
key_count = dict()
for s in keyword_occurences:
    key_count[s[0]] = s[1]
#__________________________________________________________________________
# Creation of a dictionary to replace keywords by higher frequency keywords
replacement_mot = dict()
icount = 0
for index, [mot, nb_apparitions] in enumerate(keyword_occurences):
    if nb_apparitions > 5: continue  # only the keywords that appear less than 5 times
    lemma = get_synonyms(mot)
    if len(lemma) == 0: continue     # case of the plurals
    #_________________________________________________________________
    list_mots = [(s, key_count[s]) for s in lemma 
                  if test_keyword(s, key_count, key_count[mot])]
    list_mots.sort(key = lambda x:(x[1],x[0]), reverse = True)    
    if len(list_mots) <= 1: continue       # no replacement
    if mot == list_mots[0][0]: continue    # replacement by himself
    icount += 1
    if  icount < 8:
        print('{:<12} -> {:<12} (init: {})'.format(mot, list_mots[0][0], list_mots))    
    replacement_mot[mot] = list_mots[0][0]

print(90*'_'+'\n'+'The replacement concerns {}% of the keywords.'
      .format(round(len(replacement_mot)/len(keywords)*100,2)))

fusion       -> merger       (init: [('merger', 2), ('fusion', 1)])
coke         -> snow         (init: [('snow', 74), ('coke', 1), ('blow', 1)])
pity         -> shame        (init: [('shame', 4), ('pity', 1)])
siddhartha   -> buddha       (init: [('buddha', 5), ('siddhartha', 1)])
solitary     -> hermit       (init: [('hermit', 11), ('recluse', 8), ('solitary confinement', 4), ('troglodyte', 1), ('solitary', 1)])
board        -> circuit board (init: [('circuit board', 2), ('card', 2), ('board', 1)])
jack         -> seaman       (init: [('seaman', 3), ('seafarer', 2), ('jackass', 2), ('jack', 1)])
__________________________________________________________________________________________
The replacement concerns 4.77% of the keywords.


In [27]:
# 2 successive replacements
#---------------------------
print('Keywords that appear both in keys and values:'.upper()+'\n'+45*'-')
icount = 0
for s in replacement_mot.values():
    if s in replacement_mot.keys():
        icount += 1
        if icount < 10: print('{:<20} -> {:<20}'.format(s, replacement_mot[s]))

for key, value in replacement_mot.items():
    if value in replacement_mot.keys():
        replacement_mot[key] = replacement_mot[value]          

KEYWORDS THAT APPEAR BOTH IN KEYS AND VALUES:
---------------------------------------------
tip                  -> wind                
successor            -> heir                
smell                -> spirit              
drive                -> crusade             
force                -> violence            
loot                 -> sugar               
tail                 -> shadow              
risk                 -> danger              
hole                 -> trap                


In [28]:
df_keywords_synonyms = \
            replacement_df_keywords(df_keywords_cleaned, replacement_mot, roots = False)   
keywords, keywords_roots, keywords_select = \
            keywords_inventory(df_keywords_synonyms, column = 'keywords')

Nb of keywords in variable 'keywords': 17972


In [29]:
new_keyword_occurences, keywords_count = count_word(df_keywords_synonyms,
                                                    'keywords',keywords)
new_keyword_occurences[1:6]

[['woman director', 2699],
 ['independent film', 1865],
 ['murder', 1308],
 ['music', 907],
 ['based on novel', 817]]

In [30]:
df_keywords_occurence = \
    replacement_df_low_frequency_keywords(df_keywords_synonyms, new_keyword_occurences)
keywords, keywords_roots, keywords_select = \
    keywords_inventory(df_keywords_occurence, column = 'keywords')   

Nb of keywords in variable 'keywords': 6186


In [31]:
#-------------------
new_keyword_occurences, keywords_count = count_word(df_keywords_occurence,
                                                    'keywords',keywords)
new_keyword_occurences[1:6]

[['woman director', 2699],
 ['independent film', 1865],
 ['murder', 1308],
 ['music', 907],
 ['based on novel', 817]]

In [32]:
dum = find_similarities(df, 0, del_sequels = True, verbose = True)

__________________________________________________________________________________________
QUERY: films similar to id=0 -> 'Toy Story'
nº1      -> Monsters, Inc.                
nº2      -> Heathcliff: The Movie         
nº3      -> I Want a Dog for Christmas, Charlie Brown
